In [1]:
import tensorflow as tf
import numpy as np

2025-04-14 19:28:23.837662: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-14 19:28:24.306191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744658904.532399    2410 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744658904.575578    2410 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 19:28:25.136657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
tf.__version__

'2.18.0'

In [3]:
from scipy import misc
import os

 # ----------------------------
 # Parsing function with standard python:

def zip_data_parser(zip_fname):
    os.system('unzip {0}'.format(zip_fname)) # unzip
    folder_name = zip_fname.rsplit('.zip')[0]

    # load data:
    x_stack = []
    y_stack = []
    for i in range(n_images):
        x_stack.append(misc.imread(folder_name + '/image-{0}.png'.format(i)))
        y_stack.append(misc.imread(folder_name + '/mask-{0}.png'.format(i)))
    x = np.array(x_stack)
    y = np.array(y_stack)

    os.system('rm -rf {0}'.format(folder_name)) # remove unzipped folder
    return x, y

# ----------------------------
# Dataset pipeline:
num_threads = 1
b_size = 10

all_zip_paths = ['../data/'] # list of paths for each zip file
train_data = tf.constant(all_zip_paths)
train_data = tf.data.Dataset.from_tensor_slices(train_data)

train_data = train_data.map(
        lambda filename: tf.compat.v1.py_func(  # Parse the record into tensors
            zip_data_parser,
            [filename],
            [tf.float32, tf.float32]), num_parallel_calls=num_threads)

# un-batch first, then batch the data again to have dimension [batch_size, N, M, C]
train_data = train_data.apply(tf.data.experimental.unbatch())
train_data = train_data.batch(b_size, drop_remainder=True)

2025-04-14 19:28:34.225422: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


AttributeError: in user code:

    File "/tmp/ipykernel_2410/1014688677.py", line 33, in None  *
        [tf.float32, tf.float32])

    AttributeError: module 'tensorflow' has no attribute 'py_func'


In [12]:
print('Loading data...\n')
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print('MNIST dataset loaded.\n')

Loading data...

MNIST dataset loaded.



In [13]:
x_train = x_train/255.0

In [16]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [17]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
print('Training model...\n')
model.fit(x_train, y_train, epochs=3, batch_size=32)

print('Model trained successfully!')

Training model...

Epoch 1/3
1875/1875 [==============================] - 8s 3ms/step - loss: 0.4426 - accuracy: 0.8755
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2481 - accuracy: 0.9295
Epoch 3/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2100 - accuracy: 0.9396
Model trained successfully!
